In [1]:
import requests
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

Downloading:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [3]:
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=1,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return res[0]

In [9]:
def summary(job_description, num_highlights=5):
    tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
    model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")

    input_ids = tokenizer.encode("summarize: " + job_description, return_tensors="pt", truncation=True)
    outputs = model.generate(input_ids=input_ids, max_length=100, min_length=30, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    highlights = [highlight.strip() for highlight in summary.split('\n') if highlight.strip()][:num_highlights]
    html_string = "<ul>"
    for highlight in highlights:
        html_string += f"<li>{highlight}</li>"
    html_string += "</ul>"
    return html_string

In [43]:
def summary(job_description):
    tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
    model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
    
    input_ids = tokenizer.encode("summarize: " + remove_tags(job_description), return_tensors="pt", truncation=True)
    outputs = model.generate(input_ids=input_ids, max_length=300, min_length=30, num_beams=5, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    highlights = [highlight.strip() for highlight in summary.split('\n') if highlight.strip()]
    html_string = ""
    for highlight in highlights:
        html_string += f"{highlight}"
    return html_string

In [44]:
desc="<p>***We pay Mileage Reimbursement! ***</p><p>Ask about our reimbursement program.</p><p>Job Description:  Respiratory Therapists are a valuable member of our health care team who are responsible for coordinating and providing direct respiratory therapy to the residents.</p><p>About us: <b>Camelot Care Center</b> is a skilled pediatric nursing facility located in Logansport, Indiana. <b>Camelot Care Center</b> is seeking compassionate Respiratory Therapists to become a part of our team where our primary goal is to provide exceptional care to our residents. </p><p>About the Role: <b>Camelot Care Center</b> is seeking Respiratory Therapists in our skilled nursing facilities to</p><ul><li>Provide routine respiratory therapy as well as trach care, trach changes, high flow heated humidification, Cough Assist, mechanical vest therapy and ventilator management.</li><li>Ensure the quality of resident care meets or exceeds standards of practice, while maintaining compliance with state and federal regulations.</li><li>Demonstrate flexibility in adjusting to change of residents' needs.</li><li>Exhibit courtesy, compassion, and respect to residents, families, visitors, physicians and co-workers.</li><li>Promote the residents ability to remain as independent as possible offering praise and encouragement</li><li>Demonstrate excellent leadership abilities, demonstrating a willingness to take risks, generate new ideas, evaluate and recognize priorities, keep current on information, and foster high performance, recognize a need for and provide adequate resources.</li></ul><p>About you:  The ideal candidate would have the following skills and experience.</p><ul><li>Must be licensed in the state of Indiana as a Registered or Certified Respiratory Therapist or Registered or Licensed Practical Nurse.</li><li>One year of clinical experience preferred.</li><li>Current BLS CPR certification.</li><li>Must display good emotional health and be capable of extended periods of standing, bending and heavy lifting. </li><li>Must show knowledge and understanding of regulatory standards and compliance requirements.</li><li>Ability to maintain professional attitude, including such areas as confidentiality, concern for all people, cooperativeness and conscientious use of time. </li></ul><p>We'd love for you to grow with us! <b>Camelot Care Center</b> offers:</p><ul><li>Tuition Assistance Programs</li><li>Student loan Repayment Program</li><li>Career advancement Opportunities: We'd love to help train and get you certified into a new career!</li></ul><p><b>Camelot Care Center</b> offers<strong> the above and many other benefits and perks!</strong></p><p>If you are ready to join the <b>Camelot Care Center</b> Family, please apply online today!</p><p><b>Camelot Care Center</b> is an equal opportunity employer and gives employment and promotional consideration without regard to race, color, sex, religion, age, national origin, marital or veteran status, disability, sexual orientation, gender identity, or any other protected class as defined by local, state, or federal law.</p>"

In [45]:
summary(desc)

"***We pay Mileage Reimbursement! ***Ask about our reimbursement program! ***We pay Mileage Reimbursement! ***Ask about our reimbursement program! We're seeking Respiratory Therapists to become a part of our team who are responsible for providing direct respiratory therapy to our residents. Ensure the quality of resident care meets or exceeds standards of practice. Must be licensed in the state of Indiana as a Registered or Certified Respiratory Therapist or Registered or Licensed Practical Nurse."

In [5]:
def get_Job_highlights(desc):
    res=requests.get('http://3.216.49.19:8042/get_classification/',params={'seed':desc})
    desc_highlights=json.loads(res.text)
    if len(desc_highlights)>=1:
        result=dict()
        if len(desc_highlights[1]['jobHighlights'])>1:
            for highlights in desc_highlights[1]['jobHighlights']:
                prediction=highlights['prediction']
                if prediction in result.keys():
                    if highlights['propability']>=0.95:
                        result[prediction].append(highlights['uncleaned'].capitalize())
                else:
                    if highlights['propability']>=0.95:
                        result[prediction]=[]
                        result[prediction].append(highlights['uncleaned'].capitalize())
            content='<b>Job highlights :</b><br>'
            for key,values in result.items():
                content+='<b>'+key+'</b>'
                content+='<p><ul>'
                for i in values:
                    content+='<li>'+i+'</li>'
                content+='</ul></p>'
            return content
        else:
            return
    else:
        return

In [14]:
import re
def remove_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [7]:
def mod_desc(org_desc):
    clean_desc = remove_tags(org_desc)
    summary_res = summary(clean_desc)
    Job_highlights=get_Job_highlights(org_desc)
    summary_paraphrase = paraphrase(summary_res)
    summary_paraphrase = "<b>summary: </b>"+'<p>'+summary_paraphrase+'</p>'
    mod_desc =Job_highlights.strip()+'<br>'+'<b>Description :</b>'+org_desc.strip()+'<br>'+summary_paraphrase.strip()
    return mod_desc

In [8]:
org_desc = """
<div data-test-component="StencilText"><strong>Type: </strong>Part Time</div><div data-test-component="StencilText"><strong>Duration: </strong>Seasonal</div><div data-test-component="StencilText"><strong>Pay Rate: </strong>Up to $17.5</div>  <div><div><h3>Amazon Sortation Center Warehouse Associate</h3></div><div><strong>Job Overview</strong><br>You'll be part of the dedicated Amazon team that is the first stop on the journey from the warehouse. Our fast-paced, physical roles take place in a sortation center. You will be sorting, scanning, and stacking packages on pallets, helping to get customer orders ready for delivery. Because most of the shifts are part-time, you are in control of your schedule — choose shifts that work best for you now and adjust your schedule in the future, as needed.</div><div>Duties &amp; Responsibilities</div><div><ol><div>Some of your duties may include:</div><div><ol><li>Receive and sort inventory</li><li>Get customer orders ready for delivery</li><li>Use technology like scanners, computers, and handheld printers</li></ol></div></ol></div><div><div><ol><div>You'll also need to be able to:</div><div><ol><li>Lift up to 49 pounds</li><li>Stand, walk, push, pull, squat, bend, and reach during shifts</li><li>Use carts, dollies, hand trucks, and other gear to move items around</li><li>Go up and down stairs (where applicable)</li><li>Work at a height of up to 40 feet on a mezzanine (where applicable)</li></ol></div><div>What it's like at an Amazon Sortation Center</div><div><ol><li><strong>Safety</strong>. Your safety is important to us, so we provide protective gear. All teams share safety tips daily.</li><li><strong>Surroundings</strong>. You'll be working around moving machines – order pickers, stand-up forklifts, turret trucks, and mobile carts.</li><li><strong>Activity</strong>. Some activities may require standing in one place for long periods, walking around, or climbing stairs.</li><li><strong>Temperature</strong>. Even with climate controls, temperatures can vary between 60°F  and 90°F in some parts of the warehouse; on hot days, temperatures can be over 90°F in the truck yard or inside trailers.</li><li><strong>Noise level.</strong> It can get noisy at times. We provide hearing protection if you need it.</li><li><strong>Dress code. </strong>Relaxed, with a few rules to follow for safety. Comfortable, closed-toe shoes are required and protective safety footwear are required in select business units. Depending on the role or location, Amazon provides a $110 Zappos gift code towards the purchase of shoes, in order to have you prepared for your first day on the job.</li></ol></div><div><strong>Why You'll Love this Job</strong></div><div><ol><li><strong>Stay busy.</strong> You and your team are engaged the entire shift.</li><li><strong>Schedule flexibility</strong>. Depending on where you work, schedules may include full-time (40 hours), reduced-time (30-36 hours) or part-time (20 hours or less), all with the option of working additional hours if needed. Learn more about our schedules.</li><li><strong>Shift options</strong>. Work when it works for you. Shifts may include overnight, early morning, day, evening, and weekend. You can even have four-day workweeks and three-day weekends. Find out more about our shifts.</li><li><strong>Anytime Pay.</strong> You can instantly cash out up to 70% of your earnings immediately after your shift (for select employee groups). Learn more about Anytime Pay.</li><li><strong>Career advancement.</strong> We have made a pledge to upskill our employees and offer a variety of free training and development programs, and we also have tuition support options for select employee groups. See where your Amazon journey can take you.</li><li><strong>New skills.</strong> Depending on the role and location, you'll learn how to use the latest Amazon technology – including handheld devices and robotics.</li><li><strong>Team environment.</strong> Work on small or large teams that support each other.</li></ol></div><div><strong>Why You'll Love Amazon </strong><br>Many of our jobs come with great benefits – including healthcare, parental leave, ways to save for the future, and opportunities for career advancement – all in a safe and inclusive environment that's been ranked among the best workplaces in the world. Some offerings are dependent upon the role, work schedule, or location, and can include the following:</div><div><ol><li>Competitive wage paid weekly, with overtime for more than 40 hours</li><li>Healthcare (medical, dental, vision, prescription drugs)</li><li>Medical Advice Line</li><li>Network of Support (health &amp; well-being on and off the job)</li><li>Adoption Assistance</li><li>Maternity and Parental Leave</li><li>401(k) savings plan</li><li>Paid Time Off (PTO)</li><li>Holiday pay opportunities</li><li>Employee discounts</li><li>Basic life insurance</li><li>AD&amp;D insurance</li><li>Company-paid Short-Term and Long-Term Disability</li><li>On-the-job training and skills development</li><li>Employee Assistance Program</li></ol></div><div>Learn more about Amazon Benefits and Culture.</div><div><strong>Requirements:</strong></div><div><ol><li>Candidates must be 18 years or older with the ability to understand and adhere to all job requirement and safety guidelines</li></ol></div></ol></div><div><strong>How To Get Started</strong><br>You can begin by applying above. If you need help with your application or to learn more about our hiring process, you can find support here: https://hiring.amazon.com/hiring-process#/.<br><br>If you have a disability and need an accommodation during the application and hiring process, including support for the New Hire Event, or need to initiate a request prior to starting your Day 1, please visit https://hiring.amazon.com/people-with-disabilities#/ or contact the Applicant-Candidate Accommodation Team (ACAT). You can reach us by phone at 888-435-9287, Monday through Friday, between 6 a.m. and 4 p.m. PT. <br><br>Amazon is committed to a diverse and inclusive workplace. Amazon is an equal opportunity employer and does not discriminate on the basis of race, national origin, gender, gender identity, sexual orientation, protected veteran status, disability, age, or other legally protected status.</div></div></div> PandoLogic. Keywords: Warehouse Worker, Location: Cashion, AZ - 85329"""


In [9]:
res = mod_desc(org_desc)
res

C:\Users\gmani\AppData\Roaming\Python\Python39\site-packages\transformers\generation\beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


'<b>Job highlights :</b><br><b>Qualifications</b><p><ul><li>Candidates must be 18 years or older with the ability to understand and adhere to all job requirement and safety guidelines</li></ul></p><b>Benefits</b><p><ul><li>Company-paid short-term and long-term disability</li><li>We have made a pledge to upskill our employees and offer a variety of free training and development programs and we also have tuition support options for select employee groups</li><li>Holiday pay opportunities</li><li>Competitive wage paid weekly with overtime for more than 40 hours</li><li>Paid time off (pto)</li></ul></p><br><b>Description :</b><div data-test-component="StencilText"><strong>Type: </strong>Part Time</div><div data-test-component="StencilText"><strong>Duration: </strong>Seasonal</div><div data-test-component="StencilText"><strong>Pay Rate: </strong>Up to $17.5</div>  <div><div><h3>Amazon Sortation Center Warehouse Associate</h3></div><div><strong>Job Overview</strong><br>You\'ll be part of the

In [60]:
data=pd.read_csv(r'Amazon_feed(pandologic).csv')

In [61]:
data['location']=data['city']+', '+data['state']

In [62]:
data['unique_key']=data.location+data.company+data.description
data

,_id,publisher,title,location,education,experience,city,state,postalCode,country,...,expStatus,uniqueKey,processId,original_title,original_company,main_uniqueKey,miles,feedData,createdDt,unique_key
0,643f75dc86c1902735e758e0,NaN,Amazon Packer - Multiple Shifts Available,"Litchfield Park, AZ",NaN,NaN,Litchfield Park,AZ,85340,US,...,0,amazon`available`multiple`packer`shifts~~litch...,643f72f886c1902735e757e2,amazon packer - multiple shifts available,Amazon Warehouse,amazon`available`multiple`packer`shifts~~litch...,0.0,1,2023-04-19T05:02:11.011Z,"Litchfield Park, AZAmazon Warehouse&lt;div dat..."
1,643f75dc86c1902735e758e1,NaN,Amazon Picker/Packer - Overnight Shifts Available,"Tallevast, FL",NaN,NaN,Tallevast,FL,34270,US,...,0,amazon`available`overnight`pickerpacker`shifts...,643f72f886c1902735e757e2,amazon picker/packer - overnight shifts available,Amazon Warehouse,amazon`available`overnight`pickerpacker`shifts...,0.0,1,2023-04-19T05:02:11.012Z,"Tallevast, FLAmazon Warehouse&lt;div data-test..."
2,643f75dc86c1902735e758e2,NaN,Warehouse Handler - Part-time Weekend Shifts A...,"Sarasota, FL",NaN,NaN,Sarasota,FL,34243,US,...,0,available`handler`parttime`shifts`warehouse`we...,643f72f886c1902735e757e2,warehouse handler - part-time weekend shifts a...,Amazon Warehouse,available`handler`parttime`shifts`warehouse`we...,0.0,1,2023-04-19T05:02:11.012Z,"Sarasota, FLAmazon Warehouse&lt;div data-test-..."
3,643f75dc86c1902735e758e3,NaN,Amazon Picker - Immediate Hire,"Venice, FL",NaN,NaN,Venice,FL,34285,US,...,0,amazon`hire`immediate`picker~~venice~~fl~~us~~...,643f72f886c1902735e757e2,amazon picker - immediate hire,Amazon Warehouse,amazon`hire`immediate`picker~~venice~~fl~~us~~...,0.0,1,2023-04-19T05:02:11.013Z,"Venice, FLAmazon Warehouse&lt;div data-test-co..."
4,643f75dc86c1902735e758e4,NaN,Amazon Delivery Station Warehouse Associate,"Fort Ogden, FL",NaN,NaN,Fort Ogden,FL,34267,US,...,0,amazon`associate`delivery`station`warehouse~~f...,643f72f886c1902735e757e2,amazon delivery station warehouse associate,Amazon Warehouse,amazon`associate`delivery`station`warehouse~~f...,0.0,1,2023-04-19T05:02:11.014Z,"Fort Ogden, FLAmazon Warehouse&lt;div data-tes..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207,643f75dc86c1902735e7617f,NaN,"Warehouse Worker - Flexible Shifts with $3,000...","Springfield, MA",NaN,NaN,Springfield,MA,1104,US,...,0,3000`bonus`flexible`shifts`signon`warehouse`wi...,643f72f886c1902735e757e2,"warehouse worker - flexible shifts with $3,000...",Amazon Warehouse,3000`bonus`flexible`shifts`signon`warehouse`wi...,0.0,1,2023-04-19T05:02:20.147Z,"Springfield, MAAmazon Warehouse&lt;div data-te..."
2208,643f75dc86c1902735e76180,NaN,Warehouse Worker - With Sign-On Bonus!,"Hartford, CT",NaN,NaN,Hartford,CT,6103,US,...,0,bonus`signon`warehouse`with`worker~~hartford~~...,643f72f886c1902735e757e2,warehouse worker - with sign-on bonus!,Amazon Warehouse,bonus`signon`warehouse`with`worker~~hartford~~...,0.0,1,2023-04-19T05:02:20.162Z,"Hartford, CTAmazon Warehouse&lt;div data-test-..."
2209,643f75dc86c1902735e76181,NaN,"Warehouse Worker - Up To $3,000 Sign-On Bonus","North Canton, CT",NaN,NaN,North Canton,CT,6059,US,...,0,3000`bonus`signon`to`up`warehouse`worker~~cant...,643f72f886c1902735e757e2,"warehouse worker - up to $3,000 sign-on bonus",Amazon Warehouse,3000`bonus`signon`to`up`warehouse`worker~~cant...,0.0,1,2023-04-19T05:02:20.164Z,"North Canton, CTAmazon Warehouse&lt;div data-t..."
2210,643f75dc86c1902735e76182,NaN,Warehouse Worker - Flexible Shifts (19hr/week),"Nashua, NH",NaN,NaN,Nashua,NH,3063,US,...,0,19hrweek`flexible`shifts`warehouse`worker~~nas...,643f72f886c1902735e757e2,warehouse worker - flexible shifts (19hr/week),Amazon Warehouse,19hrweek`flexible`shifts`warehouse`worker~~nas...,0.0,1,2023-04-19T05:02:20.167Z,"Nashua, NHAmazon Warehouse&lt;div data-test-co..."


In [63]:
df=data.drop_duplicates(subset=['title','location','company'])

In [64]:
df

,_id,publisher,title,location,education,experience,city,state,postalCode,country,...,expStatus,uniqueKey,processId,original_title,original_company,main_uniqueKey,miles,feedData,createdDt,unique_key
0,643f75dc86c1902735e758e0,NaN,Amazon Packer - Multiple Shifts Available,"Litchfield Park, AZ",NaN,NaN,Litchfield Park,AZ,85340,US,...,0,amazon`available`multiple`packer`shifts~~litch...,643f72f886c1902735e757e2,amazon packer - multiple shifts available,Amazon Warehouse,amazon`available`multiple`packer`shifts~~litch...,0.0,1,2023-04-19T05:02:11.011Z,"Litchfield Park, AZAmazon Warehouse&lt;div dat..."
1,643f75dc86c1902735e758e1,NaN,Amazon Picker/Packer - Overnight Shifts Available,"Tallevast, FL",NaN,NaN,Tallevast,FL,34270,US,...,0,amazon`available`overnight`pickerpacker`shifts...,643f72f886c1902735e757e2,amazon picker/packer - overnight shifts available,Amazon Warehouse,amazon`available`overnight`pickerpacker`shifts...,0.0,1,2023-04-19T05:02:11.012Z,"Tallevast, FLAmazon Warehouse&lt;div data-test..."
2,643f75dc86c1902735e758e2,NaN,Warehouse Handler - Part-time Weekend Shifts A...,"Sarasota, FL",NaN,NaN,Sarasota,FL,34243,US,...,0,available`handler`parttime`shifts`warehouse`we...,643f72f886c1902735e757e2,warehouse handler - part-time weekend shifts a...,Amazon Warehouse,available`handler`parttime`shifts`warehouse`we...,0.0,1,2023-04-19T05:02:11.012Z,"Sarasota, FLAmazon Warehouse&lt;div data-test-..."
3,643f75dc86c1902735e758e3,NaN,Amazon Picker - Immediate Hire,"Venice, FL",NaN,NaN,Venice,FL,34285,US,...,0,amazon`hire`immediate`picker~~venice~~fl~~us~~...,643f72f886c1902735e757e2,amazon picker - immediate hire,Amazon Warehouse,amazon`hire`immediate`picker~~venice~~fl~~us~~...,0.0,1,2023-04-19T05:02:11.013Z,"Venice, FLAmazon Warehouse&lt;div data-test-co..."
4,643f75dc86c1902735e758e4,NaN,Amazon Delivery Station Warehouse Associate,"Fort Ogden, FL",NaN,NaN,Fort Ogden,FL,34267,US,...,0,amazon`associate`delivery`station`warehouse~~f...,643f72f886c1902735e757e2,amazon delivery station warehouse associate,Amazon Warehouse,amazon`associate`delivery`station`warehouse~~f...,0.0,1,2023-04-19T05:02:11.014Z,"Fort Ogden, FLAmazon Warehouse&lt;div data-tes..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207,643f75dc86c1902735e7617f,NaN,"Warehouse Worker - Flexible Shifts with $3,000...","Springfield, MA",NaN,NaN,Springfield,MA,1104,US,...,0,3000`bonus`flexible`shifts`signon`warehouse`wi...,643f72f886c1902735e757e2,"warehouse worker - flexible shifts with $3,000...",Amazon Warehouse,3000`bonus`flexible`shifts`signon`warehouse`wi...,0.0,1,2023-04-19T05:02:20.147Z,"Springfield, MAAmazon Warehouse&lt;div data-te..."
2208,643f75dc86c1902735e76180,NaN,Warehouse Worker - With Sign-On Bonus!,"Hartford, CT",NaN,NaN,Hartford,CT,6103,US,...,0,bonus`signon`warehouse`with`worker~~hartford~~...,643f72f886c1902735e757e2,warehouse worker - with sign-on bonus!,Amazon Warehouse,bonus`signon`warehouse`with`worker~~hartford~~...,0.0,1,2023-04-19T05:02:20.162Z,"Hartford, CTAmazon Warehouse&lt;div data-test-..."
2209,643f75dc86c1902735e76181,NaN,"Warehouse Worker - Up To $3,000 Sign-On Bonus","North Canton, CT",NaN,NaN,North Canton,CT,6059,US,...,0,3000`bonus`signon`to`up`warehouse`worker~~cant...,643f72f886c1902735e757e2,"warehouse worker - up to $3,000 sign-on bonus",Amazon Warehouse,3000`bonus`signon`to`up`warehouse`worker~~cant...,0.0,1,2023-04-19T05:02:20.164Z,"North Canton, CTAmazon Warehouse&lt;div data-t..."
2210,643f75dc86c1902735e76182,NaN,Warehouse Worker - Flexible Shifts (19hr/week),"Nashua, NH",NaN,NaN,Nashua,NH,3063,US,...,0,19hrweek`flexible`shifts`warehouse`worker~~nas...,643f72f886c1902735e757e2,warehouse worker - flexible shifts (19hr/week),Amazon Warehouse,19hrweek`flexible`shifts`warehouse`worker~~nas...,0.0,1,2023-04-19T05:02:20.167Z,"Nashua, NHAmazon Warehouse&lt;div data-test-co..."


In [65]:
df1=df.pivot_table(columns=['location','company','description'], aggfunc='size').reset_index()
df1['unique_key']=df1.location+df1.company+df1.description
lcd_count=df1.set_index('unique_key')[0].to_dict()

In [66]:
df=df.drop_duplicates(subset=['location','company','description'])

In [67]:
def get_count(key):
    return lcd_count[key]

In [68]:
df['lcd_count']=df.apply(lambda x:get_count(x['location']+x['company']+x['description']),axis=1)

C:\Users\gmani\AppData\Local\Temp\ipykernel_17260\611342642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lcd_count']=df.apply(lambda x:get_count(x['location']+x['company']+x['description']),axis=1)


In [69]:
df.columns

Index(['_id', 'publisher', 'title', 'location', 'education', 'experience',
       'city', 'state', 'postalCode', 'country', 'description',
       'employmentType', 'posted_at', 'clientJobId', 'company', 'category',
       'url', 'cpa', 'cpc', 'employerId', 'clientName', 'source', 'expStatus',
       'uniqueKey', 'processId', 'original_title', 'original_company',
       'main_uniqueKey', 'miles', 'feedData', 'createdDt', 'unique_key',
       'lcd_count'],
      dtype='object')

In [70]:
df['lcd_count'].sum()

1695

In [71]:
df

,_id,publisher,title,location,education,experience,city,state,postalCode,country,...,uniqueKey,processId,original_title,original_company,main_uniqueKey,miles,feedData,createdDt,unique_key,lcd_count
0,643f75dc86c1902735e758e0,NaN,Amazon Packer - Multiple Shifts Available,"Litchfield Park, AZ",NaN,NaN,Litchfield Park,AZ,85340,US,...,amazon`available`multiple`packer`shifts~~litch...,643f72f886c1902735e757e2,amazon packer - multiple shifts available,Amazon Warehouse,amazon`available`multiple`packer`shifts~~litch...,0.0,1,2023-04-19T05:02:11.011Z,"Litchfield Park, AZAmazon Warehouse&lt;div dat...",1
1,643f75dc86c1902735e758e1,NaN,Amazon Picker/Packer - Overnight Shifts Available,"Tallevast, FL",NaN,NaN,Tallevast,FL,34270,US,...,amazon`available`overnight`pickerpacker`shifts...,643f72f886c1902735e757e2,amazon picker/packer - overnight shifts available,Amazon Warehouse,amazon`available`overnight`pickerpacker`shifts...,0.0,1,2023-04-19T05:02:11.012Z,"Tallevast, FLAmazon Warehouse&lt;div data-test...",3
2,643f75dc86c1902735e758e2,NaN,Warehouse Handler - Part-time Weekend Shifts A...,"Sarasota, FL",NaN,NaN,Sarasota,FL,34243,US,...,available`handler`parttime`shifts`warehouse`we...,643f72f886c1902735e757e2,warehouse handler - part-time weekend shifts a...,Amazon Warehouse,available`handler`parttime`shifts`warehouse`we...,0.0,1,2023-04-19T05:02:11.012Z,"Sarasota, FLAmazon Warehouse&lt;div data-test-...",2
3,643f75dc86c1902735e758e3,NaN,Amazon Picker - Immediate Hire,"Venice, FL",NaN,NaN,Venice,FL,34285,US,...,amazon`hire`immediate`picker~~venice~~fl~~us~~...,643f72f886c1902735e757e2,amazon picker - immediate hire,Amazon Warehouse,amazon`hire`immediate`picker~~venice~~fl~~us~~...,0.0,1,2023-04-19T05:02:11.013Z,"Venice, FLAmazon Warehouse&lt;div data-test-co...",1
4,643f75dc86c1902735e758e4,NaN,Amazon Delivery Station Warehouse Associate,"Fort Ogden, FL",NaN,NaN,Fort Ogden,FL,34267,US,...,amazon`associate`delivery`station`warehouse~~f...,643f72f886c1902735e757e2,amazon delivery station warehouse associate,Amazon Warehouse,amazon`associate`delivery`station`warehouse~~f...,0.0,1,2023-04-19T05:02:11.014Z,"Fort Ogden, FLAmazon Warehouse&lt;div data-tes...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,643f75dc86c1902735e7614f,NaN,"Warehouse Worker - Night Shifts with $3,000 Si...","West Granby, CT",NaN,NaN,West Granby,CT,6090,US,...,3000`bonus`night`shifts`signon`warehouse`with`...,643f72f886c1902735e757e2,"warehouse worker - night shifts with $3,000 si...",Amazon Warehouse,3000`bonus`night`shifts`signon`warehouse`with`...,0.0,1,2023-04-19T05:02:19.968Z,"West Granby, CTAmazon Warehouse&lt;div data-te...",1
2161,643f75dc86c1902735e76151,NaN,"Warehouse Worker - Multiple Shifts with $3,000...","New Britain, CT",NaN,NaN,New Britain,CT,6051,US,...,3000`bonus`multiple`shifts`signon`warehouse`wi...,643f72f886c1902735e757e2,"warehouse worker - multiple shifts with $3,000...",Amazon Warehouse,3000`bonus`multiple`shifts`signon`warehouse`wi...,0.0,1,2023-04-19T05:02:19.977Z,"New Britain, CTAmazon Warehouse&lt;div data-te...",1
2168,643f75dc86c1902735e76158,NaN,Warehouse Attendant - Overnight Shifts Available,"Liverpool, NY",NaN,NaN,Liverpool,NY,13090,US,...,attendant`available`overnight`shifts`warehouse...,643f72f886c1902735e757e2,warehouse attendant - overnight shifts available,Amazon Warehouse,attendant`available`overnight`shifts`warehouse...,0.0,1,2023-04-19T05:02:19.989Z,"Liverpool, NYAmazon Warehouse&lt;div data-test...",1
2175,643f75dc86c1902735e7615f,NaN,Amazon Picker - With Sign-On Bonus!,"Farmington, CT",NaN,NaN,Farmington,CT,6034,US,...,amazon`bonus`picker`signon`with~~farmington~~c...,643f72f886c1902735e757e2,amazon picker - with sign-on bonus!,Amazon Warehouse,amazon`bonus`picker`signon`with~~farmington~~c...,0.0,1,2023-04-19T05:02:20.020Z,"Farmington, CTAmazon Warehouse&lt;div data-tes...",2


In [86]:
postings=pd.DataFrame()

In [75]:
t=data[data['unique_key']==df.iloc[1]['unique_key']].head(2)

In [76]:
t['description'].iloc[1]=mod_desc(df.iloc[1]['description'])

C:\Users\gmani\AppData\Roaming\Python\Python39\site-packages\transformers\generation\beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'strip'

In [87]:
for i,row in df.iterrows():
    if row['lcd_count']>1:
        print(data[data['unique_key']==row['unique_key']])
#     else:
#         print(row)

                         _id  publisher  \
1   643f75dc86c1902735e758e1        NaN   
8   643f75dc86c1902735e758e8        NaN   
11  643f75dc86c1902735e758eb        NaN   

                                                title       location  \
1   Amazon Picker/Packer - Overnight Shifts Available  Tallevast, FL   
8                     Warehouse Worker - Anytime Pay!  Tallevast, FL   
11    Warehouse Operator - Overnight Shifts Available  Tallevast, FL   

    education  experience       city state  postalCode country  ... expStatus  \
1         NaN         NaN  Tallevast    FL       34270      US  ...         0   
8         NaN         NaN  Tallevast    FL       34270      US  ...         0   
11        NaN         NaN  Tallevast    FL       34270      US  ...         0   

                                            uniqueKey  \
1   amazon`available`overnight`pickerpacker`shifts...   
8   anytime`pay`warehouse`worker~~tallevast~~fl~~u...   
11  available`operator`overnight`shifts`wa

                          _id  publisher  \
96   643f75dc86c1902735e75940        NaN   
214  643f75dc86c1902735e759b6        NaN   
231  643f75dc86c1902735e759c7        NaN   
244  643f75dc86c1902735e759d4        NaN   
306  643f75dc86c1902735e75a12        NaN   
333  643f75dc86c1902735e75a2d        NaN   
371  643f75dc86c1902735e75a53        NaN   
506  643f75dc86c1902735e75ada        NaN   
572  643f75dc86c1902735e75b1c        NaN   
594  643f75dc86c1902735e75b32        NaN   
614  643f75dc86c1902735e75b46        NaN   

                                                 title           location  \
96                      Amazon Packer - Immediate Hire  Lawrenceville, GA   
214        Amazon Warehouse Assistant - Immediate Hire  Lawrenceville, GA   
231   Amazon Packer - Part-time Night Shifts Available  Lawrenceville, GA   
244                       Amazon Packer - Anytime Pay!  Lawrenceville, GA   
306  Amazon Picker/Packer - Part-time Overnight Shi...  Lawrenceville, GA   
333      

                           _id  publisher  \
650   643f75dc86c1902735e75b6a        NaN   
1351  643f75dc86c1902735e75e27        NaN   

                                                  title    location  \
650   Amazon Packer - Hiring Immediately (With Sign-...  Fabius, NY   
1351                Amazon Packer - With Sign-On Bonus!  Fabius, NY   

      education  experience    city state  postalCode country  ... expStatus  \
650         NaN         NaN  Fabius    NY       13063      US  ...         0   
1351        NaN         NaN  Fabius    NY       13063      US  ...         0   

                                              uniqueKey  \
650   amazon`bonus`hiring`immediately`packer`signon`...   
1351  amazon`bonus`packer`signon`with~~fabius~~ny~~u...   

                     processId  \
650   643f72f886c1902735e757e2   
1351  643f72f886c1902735e757e2   

                                         original_title  original_company  \
650   amazon packer - hiring immediately (with sign

                           _id  publisher  \
692   643f75dc86c1902735e75b94        NaN   
694   643f75dc86c1902735e75b96        NaN   
976   643f75dc86c1902735e75cb0        NaN   
1074  643f75dc86c1902735e75d12        NaN   
1168  643f75dc86c1902735e75d70        NaN   
1277  643f75dc86c1902735e75ddd        NaN   
1320  643f75dc86c1902735e75e08        NaN   

                                                  title     location  \
692       Obrero de Almacén - ¡Amazon está contratando!  Phoenix, AZ   
694   Empleados de  Almacén - ¡Amazon está contratando!  Phoenix, AZ   
976                     Obrero de Almacén - Anytime Pay  Phoenix, AZ   
1074      Amazon Fulfillment Center Warehouse Associate  Phoenix, AZ   
1168  Amazon Packer - Full-time Weekend Shifts Avail...  Phoenix, AZ   
1277                Warehouse Team Member - Anytime Pay  Phoenix, AZ   
1320                     Amazon Packer - Immediate Hire  Phoenix, AZ   

      education  experience     city state  postalCode country

                           _id  publisher  \
779   643f75dc86c1902735e75beb        NaN   
947   643f75dc86c1902735e75c93        NaN   
1032  643f75dc86c1902735e75ce8        NaN   
1080  643f75dc86c1902735e75d18        NaN   
1099  643f75dc86c1902735e75d2b        NaN   
1265  643f75dc86c1902735e75dd1        NaN   
1287  643f75dc86c1902735e75de7        NaN   

                                                  title      location  \
779                                  Selector de Amazon  Glendale, AZ   
947       Amazon Fulfillment Center Warehouse Associate  Glendale, AZ   
1032    Trabajador de  Almacén - Gana hasta $17.00/hora  Glendale, AZ   
1080       Empacador de Amazon - Gana hasta $17.00/hora  Glendale, AZ   
1099                                      Amazon Picker  Glendale, AZ   
1265     Selector de Amazon - Varios turnos disponibles  Glendale, AZ   
1287  Selector de Amazon - Turnos de toda la noche d...  Glendale, AZ   

      education  experience      city state  postalCod

                           _id  publisher  \
1449  643f75dc86c1902735e75e89        NaN   
1486  643f75dc86c1902735e75eae        NaN   
1641  643f75dc86c1902735e75f49        NaN   
1700  643f75dc86c1902735e75f84        NaN   
1715  643f75dc86c1902735e75f93        NaN   
1727  643f75dc86c1902735e75f9f        NaN   
1787  643f75dc86c1902735e75fdb        NaN   
1844  643f75dc86c1902735e76014        NaN   
1876  643f75dc86c1902735e76034        NaN   
1958  643f75dc86c1902735e76086        NaN   
1990  643f75dc86c1902735e760a6        NaN   
2021  643f75dc86c1902735e760c5        NaN   
2144  643f75dc86c1902735e76140        NaN   
2189  643f75dc86c1902735e7616d        NaN   

                                                 title   location  education  \
1449                   Warehouse Worker - Anytime Pay!  Derry, NH        NaN   
1486       Amazon Warehouse Assistant - Immediate Hire  Derry, NH        NaN   
1641  Amazon Order Filler - Overnight Shifts Available  Derry, NH        NaN   
1700

                           _id  publisher  \
1510  643f75dc86c1902735e75ec6        NaN   
1559  643f75dc86c1902735e75ef7        NaN   
1572  643f75dc86c1902735e75f04        NaN   
1696  643f75dc86c1902735e75f80        NaN   
1774  643f75dc86c1902735e75fce        NaN   
1795  643f75dc86c1902735e75fe3        NaN   
2027  643f75dc86c1902735e760cb        NaN   
2097  643f75dc86c1902735e76111        NaN   
2193  643f75dc86c1902735e76171        NaN   

                                                  title    location  \
1510               Warehouse Operator - Up to $15.50/hr  Durham, NC   
1559                     Warehouse Worker - Anytime Pay  Durham, NC   
1572  Amazon Picker - Part-time Weekday Shifts Avail...  Durham, NC   
1696        Amazon Sortation Center Warehouse Associate  Durham, NC   
1774                                 Warehousing Worker  Durham, NC   
1795          Amazon Packer - Multiple Shifts Available  Durham, NC   
2027  Warehouse Worker - Part-time Night Shifts Avai

In [93]:
for i,row in df.iterrows():
    if row['lcd_count']==1:
        postings=postings.append(row.drop(['unique_key','lcd_count']))
    else:
        

In [94]:
postings

,_id,publisher,title,location,education,experience,city,state,postalCode,country,...,source,expStatus,uniqueKey,processId,original_title,original_company,main_uniqueKey,miles,feedData,createdDt
0,643f75dc86c1902735e758e0,NaN,Amazon Packer - Multiple Shifts Available,"Litchfield Park, AZ",NaN,NaN,Litchfield Park,AZ,85340,US,...,https://vendors.pandologic.com/Jobiak_A/Jobiak...,0,amazon`available`multiple`packer`shifts~~litch...,643f72f886c1902735e757e2,amazon packer - multiple shifts available,Amazon Warehouse,amazon`available`multiple`packer`shifts~~litch...,0.0,1,2023-04-19T05:02:11.011Z
3,643f75dc86c1902735e758e3,NaN,Amazon Picker - Immediate Hire,"Venice, FL",NaN,NaN,Venice,FL,34285,US,...,https://vendors.pandologic.com/Jobiak_A/Jobiak...,0,amazon`hire`immediate`picker~~venice~~fl~~us~~...,643f72f886c1902735e757e2,amazon picker - immediate hire,Amazon Warehouse,amazon`hire`immediate`picker~~venice~~fl~~us~~...,0.0,1,2023-04-19T05:02:11.013Z
4,643f75dc86c1902735e758e4,NaN,Amazon Delivery Station Warehouse Associate,"Fort Ogden, FL",NaN,NaN,Fort Ogden,FL,34267,US,...,https://vendors.pandologic.com/Jobiak_A/Jobiak...,0,amazon`associate`delivery`station`warehouse~~f...,643f72f886c1902735e757e2,amazon delivery station warehouse associate,Amazon Warehouse,amazon`associate`delivery`station`warehouse~~f...,0.0,1,2023-04-19T05:02:11.014Z
5,643f75dc86c1902735e758e5,NaN,Amazon Delivery Station Warehouse Associate,"Bradenton, FL",NaN,NaN,Bradenton,FL,34202,US,...,https://vendors.pandologic.com/Jobiak_A/Jobiak...,0,amazon`associate`delivery`station`warehouse~~b...,643f72f886c1902735e757e2,amazon delivery station warehouse associate,Amazon Warehouse,amazon`associate`delivery`station`warehouse~~b...,0.0,1,2023-04-19T05:02:11.014Z
6,643f75dc86c1902735e758e6,NaN,Amazon Delivery Station Warehouse Associate,"Nashua, NH",NaN,0.0,Nashua,NH,3106,US,...,https://vendors.pandologic.com/Jobiak_A/Jobiak...,0,amazon`associate`delivery`station`warehouse~~n...,643f72f886c1902735e757e2,amazon delivery station warehouse associate,Amazon Warehouse,amazon`associate`delivery`station`warehouse~~n...,0.0,1,2023-04-19T05:02:11.015Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,643f75dc86c1902735e7612f,NaN,Amazon Picker - Get Paid Instantly,"Chapel Hill, NC",NaN,NaN,Chapel Hill,NC,27599,US,...,https://vendors.pandologic.com/Jobiak_A/Jobiak...,0,amazon`get`instantly`paid`picker~~chapel`hill~...,643f72f886c1902735e757e2,amazon picker - get paid instantly,Amazon Warehouse,amazon`get`instantly`paid`picker~~chapel`hill~...,0.0,1,2023-04-19T05:02:19.827Z
2159,643f75dc86c1902735e7614f,NaN,"Warehouse Worker - Night Shifts with $3,000 Si...","West Granby, CT",NaN,NaN,West Granby,CT,6090,US,...,https://vendors.pandologic.com/Jobiak_A/Jobiak...,0,3000`bonus`night`shifts`signon`warehouse`with`...,643f72f886c1902735e757e2,"warehouse worker - night shifts with $3,000 si...",Amazon Warehouse,3000`bonus`night`shifts`signon`warehouse`with`...,0.0,1,2023-04-19T05:02:19.968Z
2161,643f75dc86c1902735e76151,NaN,"Warehouse Worker - Multiple Shifts with $3,000...","New Britain, CT",NaN,NaN,New Britain,CT,6051,US,...,https://vendors.pandologic.com/Jobiak_A/Jobiak...,0,3000`bonus`multiple`shifts`signon`warehouse`wi...,643f72f886c1902735e757e2,"warehouse worker - multiple shifts with $3,000...",Amazon Warehouse,3000`bonus`multiple`shifts`signon`warehouse`wi...,0.0,1,2023-04-19T05:02:19.977Z
2168,643f75dc86c1902735e76158,NaN,Warehouse Attendant - Overnight Shifts Available,"Liverpool, NY",NaN,NaN,Liverpool,NY,13090,US,...,https://vendors.pandologic.com/Jobiak_A/Jobiak...,0,attendant`available`overnight`shifts`warehouse...,643f72f886c1902735e757e2,warehouse attendant - overnight shifts available,Amazon Warehouse,attendant`available`overnight`shifts`warehouse...,0.0,1,2023-04-19T05:02:19.989Z


In [61]:
df=pd.read_csv('unique_data.csv')

In [62]:
df['modified_desc']=None

In [ ]:
for i in tqdm(range(len(df))):
    desc=df.iloc[i]['description']
    try:
        df['modified_desc'].iloc[i]=mod_desc(desc)
    except:
        continue

  0%|          | 1/200 [00:06<22:15,  6.71s/it]C:\Users\gmani\AppData\Roaming\Python\Python39\site-packages\transformers\generation\beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(
C:\Users\gmani\AppData\Local\Temp\ipykernel_12988\4241359196.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['modified_desc'].iloc[i]=mod_desc(desc)
  4%|▍         | 9/200 [04:01<1:13:59, 23.24s/it]

In [96]:
df_post = df[~df['modified_desc'].isna()]
df_post

,_id,jobtitle,company,extraInfo.cpa,extraInfo.cpc,extraInfo.mtlExpand,extraInfo.atlExpand,extraInfo.originalTitle,extraInfo.original_title,extraInfo.originalLocation,extraInfo.miles,extraInfo.tlExpand,extraInfo.locExpand,extraInfo.titleInLocation,extraInfo.feedData,location,description,description_text,modified_desc
0,64269135c98552097d8b0278,Amazon Sortation Center Warehouse Associate,Amazon,100,NaN,NaN,NaN,NaN,amazon sortation center warehouse associate,"Mesa, AZ, US, 85202",48.000,NaN,1.0,NaN,0,"Waddell, AZ, US","<div data-test-component=""StencilText""><strong...",Type: Part Time\nDuration: Seasonal\nPay Rate:...,<b>Job highlights :</b><br><b>Responsibilities...
1,642691351125e609542feb1d,Empacador de Amazon - Contratación inmediata,Amazon,200,NaN,NaN,NaN,NaN,empacador de amazon - contratación inmediata,"Paradise Valley, AZ, US, 85253",20.387,NaN,1.0,NaN,0,"Tolleson, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Full Time, Reduced Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
2,642691396ca76109687cc513,Amazon Picker - Anytime Pay,Amazon,70,NaN,NaN,NaN,NaN,amazon picker - anytime pay,"Scottsdale, AZ, US, 85257",45.118,NaN,1.0,NaN,0,"Buckeye, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Full Time, Reduced Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
3,6426913ca45d1e096e05ecb9,Selector de Amazon,Amazon,200,NaN,NaN,NaN,NaN,selector de amazon,"Tempe, AZ, US, 85287",22.640,NaN,1.0,NaN,0,"Avondale, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Full Time, Reduced Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
4,6426914f6ca76109687cc516,Amazon Packer - Multiple Shifts Available,Amazon,70,NaN,NaN,NaN,NaN,amazon packer - multiple shifts available,"Tempe, AZ, US, 85287",26.964,NaN,1.0,NaN,0,"Sun City, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Full Time, Reduced Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,642be93a81b09408deef6e5a,Selector de Amazon - Varios turnos disponibles,Amazon,200,NaN,NaN,NaN,NaN,selector de amazon - varios turnos disponibles,"Paradise Valley, AZ, US, 85253",51.723,NaN,1.0,NaN,0,"Superior, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Reduced Time, Full Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
77,642be94a391c92090a0715bb,Trabajador de Almacén - ¡Amazon está contrata...,Amazon,200,NaN,NaN,NaN,NaN,trabajador de almacén - ¡amazon está contrata...,"Phoenix, AZ, US, 85007",55.388,NaN,1.0,NaN,0,"Arizona City, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Reduced Time, Full Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
78,642be95d869b4b0903c949de,Controlador de Paquetes de Amazon - Contrataci...,Amazon,200,NaN,NaN,NaN,NaN,controlador de paquetes de amazon - contrataci...,"Phoenix, AZ, US, 85032",55.493,NaN,1.0,NaN,0,"Wickenburg, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Reduced Time, Full Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
79,642be962391c92090a0715ff,Controlador de Paquetes de Amazon - Anytime Pay,Amazon,200,NaN,NaN,NaN,NaN,controlador de paquetes de amazon - anytime pay,"Glendale, AZ, US, 85302",42.000,NaN,1.0,NaN,0,"Lower Santan Village, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Reduced Time, Full Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...


In [97]:
df_post.to_csv("post_data.csv",index=False)

In [99]:
t=df_post[['description','modified_desc']]

In [100]:
t.to_csv("test_desc.csv",index=False)

In [11]:
df_post = pd.read_csv("post_data.csv")

In [13]:
df_post = df_post.drop_duplicates(subset=['modified_desc'])

,_id,jobtitle,company,extraInfo.cpa,extraInfo.cpc,extraInfo.mtlExpand,extraInfo.atlExpand,extraInfo.originalTitle,extraInfo.original_title,extraInfo.originalLocation,extraInfo.miles,extraInfo.tlExpand,extraInfo.locExpand,extraInfo.titleInLocation,extraInfo.feedData,location,description,description_text,modified_desc
0,64269135c98552097d8b0278,Amazon Sortation Center Warehouse Associate,Amazon,100,NaN,NaN,NaN,NaN,amazon sortation center warehouse associate,"Mesa, AZ, US, 85202",48.000,NaN,1.0,NaN,0,"Waddell, AZ, US","<div data-test-component=""StencilText""><strong...",Type: Part Time\nDuration: Seasonal\nPay Rate:...,<b>Job highlights :</b><br><b>Responsibilities...
1,642691351125e609542feb1d,Empacador de Amazon - Contratación inmediata,Amazon,200,NaN,NaN,NaN,NaN,empacador de amazon - contratación inmediata,"Paradise Valley, AZ, US, 85253",20.387,NaN,1.0,NaN,0,"Tolleson, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Full Time, Reduced Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
2,642691396ca76109687cc513,Amazon Picker - Anytime Pay,Amazon,70,NaN,NaN,NaN,NaN,amazon picker - anytime pay,"Scottsdale, AZ, US, 85257",45.118,NaN,1.0,NaN,0,"Buckeye, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Full Time, Reduced Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
3,6426913ca45d1e096e05ecb9,Selector de Amazon,Amazon,200,NaN,NaN,NaN,NaN,selector de amazon,"Tempe, AZ, US, 85287",22.640,NaN,1.0,NaN,0,"Avondale, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Full Time, Reduced Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
4,6426914f6ca76109687cc516,Amazon Packer - Multiple Shifts Available,Amazon,70,NaN,NaN,NaN,NaN,amazon packer - multiple shifts available,"Tempe, AZ, US, 85287",26.964,NaN,1.0,NaN,0,"Sun City, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Full Time, Reduced Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,642be93a81b09408deef6e5a,Selector de Amazon - Varios turnos disponibles,Amazon,200,NaN,NaN,NaN,NaN,selector de amazon - varios turnos disponibles,"Paradise Valley, AZ, US, 85253",51.723,NaN,1.0,NaN,0,"Superior, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Reduced Time, Full Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
77,642be94a391c92090a0715bb,Trabajador de Almacén - ¡Amazon está contrata...,Amazon,200,NaN,NaN,NaN,NaN,trabajador de almacén - ¡amazon está contrata...,"Phoenix, AZ, US, 85007",55.388,NaN,1.0,NaN,0,"Arizona City, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Reduced Time, Full Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
78,642be95d869b4b0903c949de,Controlador de Paquetes de Amazon - Contrataci...,Amazon,200,NaN,NaN,NaN,NaN,controlador de paquetes de amazon - contrataci...,"Phoenix, AZ, US, 85032",55.493,NaN,1.0,NaN,0,"Wickenburg, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Reduced Time, Full Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
79,642be962391c92090a0715ff,Controlador de Paquetes de Amazon - Anytime Pay,Amazon,200,NaN,NaN,NaN,NaN,controlador de paquetes de amazon - anytime pay,"Glendale, AZ, US, 85302",42.000,NaN,1.0,NaN,0,"Lower Santan Village, AZ, US","<div data-test-component=""StencilText""><strong...","Type: Reduced Time, Full Time\nDuration: Seaso...",<b>Job highlights :</b><br><b>Responsibilities...
